# Day 13 - Distress Signal


## Data

In [1]:
example_data = """
[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]
""".strip()

print(example_data)

[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]


In [2]:
import aocd
raw_data = aocd.get_data(year=2022, day=13)
print(raw_data)

[[],[1],[[[1,3],2,1,3]]]
[[[],6,[3,8]],[]]

[[9,6,0],[[0]],[[[5,3,1,1,10],4,[],[10,2],[2,9,9,7]],[[10],8]],[1,[[4,9,10,0]],[[0,8],10,8],0],[[7,[5,6,5,7],9,4,[0,4]]]]
[[9,[],[10,4],[2,9,[2]]],[[[],6,3],1,7,[5,0],[7,7,[10],[0,6,2,4,7],7]]]

[[[],[],[]],[6,[[1,4,7,9],7],[]]]
[[],[],[],[[[6,2,1],[5,7],2,[8,0],[7,1,5,6]]]]

[[5,1],[3],[4,6]]
[[3,3,[1,[2,4,2,7,2],[0,3],8],9,0],[4,[6]],[],[9,[2,[9,9,2,6,7],[],2,2],[[],[10],[],[0]],6]]

[[[10,[10,3,5],1,8],[[10,4],2,10,0,6],[3,3,2,[4,4],[3,6]]],[[[],2,0,8,[2,2,3,3]]],[[8],3,10,4,4],[]]
[[1,[7,10],[[2],[10,2,1,4,2],[2,3,8],0,8],5],[7,[[10,4],8]],[],[],[]]

[[[4,[2,2,8,0]],3,5,[0],3],[4,[10],10,1],[3],[1],[1,4,[10],[[8],[5,3,0,6,6]],8]]
[[2,4],[3,1,[],[[]],[4,[7,0,3],0]],[]]

[[2,0],[[],[[3,0,6],6,2,6],8,5,[0,[10,0,10,10,8],[4,5,1]]],[[[6,7,0,6,10],[8],[1],6,7],0,6,[10,5,4,[4,2,9],0],[[2,7,8,6,7]]],[[[7,8,6,3],0,[4,3,3,10,8],[4]],[8],2,1,[1,7,[2,3,6],[7,3],9]]]
[[6],[5],[],[6,0],[[],[9,[10,5],10,[4,3,0,6,6]]]]

[10,0,2,0]
[10,0,2,0,2]

[[9],[4],

In [3]:
from dataclasses import dataclass
from typing import Callable

## Parsing

In [4]:
import pyparsing as pp

integer = pp.Word(pp.nums)
list_item = pp.Forward()
list_grammar = pp.Group(pp.delimitedList(list_item))
list_item << (integer | list_grammar)

integer = pp.Word(pp.nums)
        
@integer.set_parse_action
def parse_integer(tokens):
    return int(tokens[0])

element = pp.Forward()

list_grammar = pp.Suppress('[') + pp.Group(pp.Optional(pp.delimited_list(element))) + pp.Suppress(']')

element << (integer | list_grammar)

packet = list_grammar

@packet.set_parse_action
def parse_packet(tokens):
    return tokens.as_list()

pair = packet + packet

@pair.set_parse_action
def parse_pair(tokens):
    return tuple(tokens.as_list())


parser = pp.OneOrMore(pair)

parser.run_tests("""
[1] [2, 5, 3]

[] [[]]

[5, 3, [5, 2]] [2, 2, 2, 2]
""")


[1] [2, 5, 3]
[([1], [2, 5, 3])]

[] [[]]
[([], [[]])]

[5, 3, [5, 2]] [2, 2, 2, 2]
[([5, 3, [5, 2]], [2, 2, 2, 2])]


(True,
 [('[1] [2, 5, 3]', ParseResults([([1], [2, 5, 3])], {})),
  ('[] [[]]', ParseResults([([], [[]])], {})),
  ('[5, 3, [5, 2]] [2, 2, 2, 2]',
   ParseResults([([5, 3, [5, 2]], [2, 2, 2, 2])], {}))])

In [5]:
example_pairs = parser.parse_string(example_data).as_list()
real_pairs = parser.parse_string(raw_data).as_list()
example_pairs

[([1, 1, 3, 1, 1], [1, 1, 5, 1, 1]),
 ([[1], [2, 3, 4]], [[1], 4]),
 ([9], [[8, 7, 6]]),
 ([[4, 4], 4, 4], [[4, 4], 4, 4, 4]),
 ([7, 7, 7, 7], [7, 7, 7]),
 ([], [3]),
 ([[[]]], [[]]),
 ([1, [2, [3, [4, [5, 6, 7]]]], 8, 9], [1, [2, [3, [4, [5, 6, 0]]]], 8, 9])]

## Part 1

In [6]:
def compare_int(left, right):
    """
    Given two ints, return:
      neg if left  < right,
       0  if left == right,
      pos if left  > right.
    """
    return left - right


assert compare_int(5, 5) == 0
assert compare_int(2, 5) < 0
assert compare_int(5, 2) > 0


def compare_lists(left, right):
    """
    Given two lists, return:
      neg if left  < right,
       0  if left == right,
      pos if left  > right.
    """
    
    for i in range(min(len(left), len(right))):
        comparison = compare(left[i], right[i])
        if comparison != 0:
            return comparison
        
    return compare(len(left), len(right))


def compare(left, right):
    """
    Given two elements, return:
      neg if left  < right,
       0  if left == right,
      pos if left  > right.
    """

    left_type = type(left)
    right_type = type(right)
    
    if right_type == int and left_type == int:
        return compare_int(left, right)
    else:
        if right_type == int:
            right = [right]
            
        if left_type == int:
            left = [left]
        
        return compare_lists(left, right)
    
    
assert compare([], []) == 0
assert compare([1, 2, 3], [1, 2, 3]) == 0
assert compare([1, 2, 5], [1, 2, 3]) >  0
assert compare([2, 1, 1], [1, 2, 2]) >  0

assert compare([1, 1, 3, 1, 1], [1, 1, 5, 1, 1]) == -2
assert compare([[1],[2,3,4]], [[1],4]) < 0
assert compare([[[]]], []) > 0
assert compare([7, 7, 7], [7, 7, 7, 7]) == -1



In [11]:
def index(pair, i):
    """Returns the index of pair if it is in the correct order"""
    left, right = pair
    
    if compare(left, right) <= 0:
        return i + 1
    return 0

def sum_indices(pairs):
    """Return the sum of the indices of the pairs that are in the correct order."""
    index_sum = 0
    
    for i in range(len(pairs)):
        index_sum += index(pairs[i], i)
    return index_sum
    
assert sum_indices(example_pairs) == 13
sum_indices(real_pairs)

5588

## Part 2

NameError: name 'parse' is not defined